In [6]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langserve import RemoteRunnable
from langchain_core.runnables import RunnableParallel
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

In [3]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [4]:
from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="llama3:latest")
# llm = ChatOllama(model="Llama-3-Open-Ko-8B-FP16.gguf:latest")
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

In [27]:
llm.invoke("What is stock?")

AIMessage(content="Stock refers to shares or ownership in a company that are traded on public markets like stock exchanges. When you buy stocks, you essentially become a part-owner of the company, and your investment can grow or decline based on the company's performance and market conditions. Stocks represent an equity claim on the assets and earnings of a company, meaning that if the company is liquidated, stockholders would be paid after all debts and other liabilities have been settled.\n\nThere are two main types of stocks: common stocks and preferred stocks. Common stocks typically provide voting rights to shareholders and entitle them to receive dividends (if declared by the board of directors) and any remaining profits if the company is liquidated. Preferred stocks, on the other hand, usually do not have voting rights but offer a fixed or variable dividend payment and have a higher claim on assets in case of liquidation compared to common stocks.\n\nInvesting in stocks can be a

In [28]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

chain = prompt | llm
chain.invoke({"input": "What is stock?"})

AIMessage(content='안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 궁금한 점을 도와드리기 위해 여기에 있습니다. 주식을 간단히 설명하자면, 주식은 회사의 일부를 소유하는 것을 의미합니다. 즉, 특정 회사에 투자하여 그 회사의 성장과 성공에 기여하고, 그에 따라 수익을 얻을 수 있는 기회를 가지게 됩니다.\n\n주식은 공개 시장에서 거래되며, 이는 사람들이 서로에게 주식을 사고팔 수 있다는 의미입니다. 이러한 거래를 통해 주가는 오르거나 내릴 수 있으며, 이는 해당 회사의 성과와 시장 상황에 따라 달라집니다.\n\n주식에 투자할 때는 항상 위험이 따르므로, 결정을 내리기 전에 철저히 조사하고 본인의 재정 상황과 목표에 맞는지 확인하는 것이 중요합니다. 또한, 주식은 장기적으로 투자하는 것이 일반적으로 더 좋으므로, 인내심을 가지고 장기적인 관점을 유지하는 것이 중요합니다.\n\n더 궁금한 점이 있거나 도움이 필요하시면 언제든지 물어보세요!', response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-05-31T22:32:18.033392Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7225886333, 'load_duration': 4188000, 'prompt_eval_count': 45, 'prompt_eval_duration': 280213000, 'eval_count': 181, 'eval_duration': 6938180000}, id='run-8320c375-366c-4b5d-a15f-28362f65e44b-0')

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "What is stock?"})

'안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 궁금한 점을 도와드리겠습니다. 주식에 대해 설명해 드리겠습니다.\n\n주식은 한 회사의 소유권을 나타내는 증권으로, 투자자들이 그 회사의 일부를 소유하게 됩니다. 예를 들어, 어떤 회사가 100개의 주식을 발행했다면, 각 주식을 구매한 사람은 해당 회사의 1%를 소유하게 됩니다. 이러한 방식으로 사람들은 다양한 사업에 투자하고 수익을 창출할 수 있습니다.\n\n주식은 거래소에서 사고팔 수 있으며, 주식의 가격은 공급과 수요에 따라 변동됩니다. 또한, 기업들은 보통 매년 배당금을 지급하는데, 이는 투자자들에게 주식을 보유하고 있는 대가로 받는 수익입니다.\n\n요약하자면, 주식은 사람들이 회사의 일부를 소유하고 수익을 창출할 수 있게 해주는 금융 상품으로, 거래소에서 사고팔 수 있으며 배당금 형태로 수익을 얻을 수 있습니다.'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer me in Korean no matter what"),
    ("user", "{input}")
])

output_parser = StrOutputParser()

# chain = prompt | llm | output_parser
my_input = {"input": "What is stock?"}
print(f'1st chain >> {prompt.invoke(my_input)}')
print(f'2nd chain >> {(prompt | llm).invoke(my_input)}')
print(f'3rd chain >> {(prompt | llm | output_parser).invoke(my_input)}')

1st chain >> messages=[SystemMessage(content='You are a helpful, professional assistant named 권봇. answer me in Korean no matter what'), HumanMessage(content='What is stock?')]
2nd chain >> content='주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자들이 회사의 성장과 성공에 기여하고자 자본을 제공함으로써 취득하게 됩니다. 주식을 구매하면 해당 기업의 일부가 되며, 그 성과에 따라 배당금이나 미래 주식 가치 상승으로 인한 이익을 얻게 됩니다. 또한, 주식을 매매하여 가격 변동에 따른 차익을 실현할 수도 있습니다.' response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-05-06T03:09:00.611458Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 16761211959, 'load_duration': 6395765875, 'prompt_eval_count': 40, 'prompt_eval_duration': 979833000, 'eval_count': 79, 'eval_duration': 9379067000} id='run-23955113-0cc5-4ad0-a48a-7accd66da0de-0'
3rd chain >> 주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자가 회사의 일부분을 구매함으로써 해당 기업의 성과에 대한 재정적 이익을 얻을 수 있습니다. 주식은 공개 시장에서 거래되며, 이는 사람들이 증권 거래소나 온라인 플랫폼을 통해 주식을 사고팔 수 있음을 의미합니다. 이러한 방식으로 개인들은 다양한 산업에 걸쳐 있는 기업들에 투자할 기회를 가질 수 있으며, 이는 결국 

In [ ]:
chain = prompt | llm | output_parser
for token in chain.stream(
    {"input": "What is stock?"}
):
    print(token, end="")

주식(stu-gik)은 일반적으로 공 司의 주가를 의미하는 단어입니다. 또한 주식을 매각하거나 매입하여 이익을 얻는 투자 방식을도 나타냅니다. 다른 말로서는 주식 시장에서 거래되는 증권을 총괄하는 말입니다.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# 첫번째 체인
prompt1 = ChatPromptTemplate.from_template("[{korean_input}] translate the question into English. Don't say anything else, just translate it.")
chain1 = (
    prompt1 
    | llm 
    | StrOutputParser()
)
message1 = chain1.invoke({"korean_input": "주식이 뭐야?"})
print(f'message1: {message1}')

# 두번째 체인 (첫번째 체인 연결)
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer me in Korean no matter what"),
    ("user", "{input}")
])
chain2 = (
    {"input": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)
message2 = chain2.invoke({"korean_input": "주식이 뭐야?"})
print(f'message2: {message2}')

message1: What is a stock?
message2: 주식은 한 회사의 소유권을 나타내는 증권입니다. 주식을 구매하면 해당 회사의 일부를 소유하게 되며, 이는 회사 성과에 따른 배당금이나 미래 주식 가치 상승으로 인한 이익을 얻을 수 있는 권리를 가지게 됩니다. 간단히 말해서, 주식을 사는 것은 그 회사의 일부분을 사는 것과 같습니다.


In [14]:
joke_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 짧은 농담 말해줘") 
    | llm)
poem_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 시 2줄 써줘") 
    | llm)

# map_chain = {"joke": joke_chain, "poem": poem_chain} # 체인에서 이처럼 사용할 때, 자동으로 RunnableParallel 사용됨
# map_chain = RunnableParallel({"joke": joke_chain, "poem": poem_chain})
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "애플"})

{'joke': AIMessage(content="알고 보니 애플이 '아이스버그'를 사들였대요 - 이제 모든 아이폰에는 빙산 앱이 기본으로 설치돼 있답니다!", response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-06-07T06:11:49.386905Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1747483625, 'load_duration': 19981500, 'prompt_eval_count': 16, 'prompt_eval_duration': 379381000, 'eval_count': 36, 'eval_duration': 1345341000}, id='run-80a6ba77-c5ca-4741-a020-909537deea41-0'),
 'poem': AIMessage(content='\n아이폰의 매혹적인 화면이 우리를 사로잡네,\n맥북은 우리의 창의력을 불러일으키며,\n시리야, 언제나 도와주고, 아이패드는 우리 손끝에서 놀랍게 작동해,\n애플 워치, 멋진 액세서리, 항상 함께하지.\n\n아이팟의 음악이 우리를 춤추게 하고, 애플 TV는 엔터테인먼트를 선사해,\n맥 프로와 맥북 프로가 우리의 작업 흐름을 원활하게 만들어줘,\n애플 생태계, 우리 삶을 풍요롭게 하며, 언제나 새로운 것을 발견할 수 있게 해.\n기술의 정점, 우리가 사랑하는 브랜드, 바로 애플이야!', response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-06-07T06:11:55.114393Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
message = remote_chain.invoke({
    "input": "What is stock?",
})
print(f'message : {message}')

message : 주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자가 회사의 일부분을 구매함으로써 해당 기업의 성과에 대한 재정적 이익을 얻을 수 있습니다. 주식은 공개 시장에서 거래되며, 이는 사람들이 증권 거래소나 온라인 플랫폼을 통해 주식을 사고팔 수 있음을 의미합니다. 이러한 방식으로 개인들은 다양한 산업에 걸쳐 있는 기업들에 투자할 기회를 가질 수 있으며, 이는 결국 그들의 포트폴리오의 다양화와 위험 분산을 가능하게 합니다.
